# Making sense of snscrape

Author: Sid

In [1]:
%load_ext autoreload
%autoreload 2

Make sure snscrape is is installed, you can do this by running `pip install snscrape` in your terminal. Then run the below cell to import the required libraries.

In [2]:
import pandas as pd
from twitscrape import *
import snscrape.modules.twitter as sntwitter

Note: Make sure you are running the latest version of snscrape. run this command in your terminal just in case.

```pip install --upgrade snscrape```

## Why snscrape?
Over the recent years, Twitter API has gotten a lot worse. Most features have been paywalled, and the ones that are available are heavily rate limited so that the servers dont get overloaded. While this is a good thing in theory for stability, it isnt ideal for people who want to actually scrape tweets from twitter and analyse data about them. Using snscrape is a good way to harvest data without using the API. The only issue is, its really poorly documented in terms of python modules and object names. This notebook explores some of the different scrapers in the library.

## Looking at different modules

Some modules include:
* TwitterProfileScraper
* TwitterTweetScraper
* TwitterUserScraper
* TwitterTrendsScraper

For the purposes of the project we probably want to look at TwitterUserScraper, as it doesnt include other people mentioning the user that we are requesting the search for. 

You can look at all the contents that it gets from the most recent tweet in a profile, and from that we can grab all of the useful data we need and manipulate it however we want.

Below is a cell that will grab the data of the latest tweet from NAME, try it if you want and see all the info we can use.

In [4]:
NAME = "jack" #paste the twitter handle here without the '@'
user_scraper = sntwitter.TwitterUserScraper(NAME).get_items()
print (type(user_scraper))
for tweet in user_scraper:
    break
tweet

<class 'generator'>


Tweet(url='https://twitter.com/jack/status/1620552043609096192', date=datetime.datetime(2023, 1, 31, 22, 38, 10, tzinfo=datetime.timezone.utc), rawContent='and Google Play Store: https://t.co/1Ve7GIBG0F\n\nand of course, the open web: https://t.co/qXl9xrmtKE (one of many)', renderedContent='and Google Play Store: play.google.com/store/apps/det…\n\nand of course, the open web: snort.social (one of many)', id=1620552043609096192, user=User(username='jack', id=12, displayname='jack', rawDescription='#bitcoin\n#nostr: npub1sg6plzptd64u62a878hep2kev88swjh3tw00gjsfl8f237lmu63q0uf63m', renderedDescription='#bitcoin\n#nostr: npub1sg6plzptd64u62a878hep2kev88swjh3tw00gjsfl8f237lmu63q0uf63m', descriptionLinks=None, verified=True, created=datetime.datetime(2006, 3, 21, 20, 50, 14, tzinfo=datetime.timezone.utc), followersCount=6552139, friendsCount=4624, statusesCount=29016, favouritesCount=35950, listedCount=33492, mediaCount=2904, location='', protected=False, link=TextLink(text='snort.social/p/n

As you can see, there are a bunch of fields that the module pulled from the latest tweet. For example, `date` gives you the date and time that the tweet happened, retweetCount and likeCount give you the number of retweets and likes that a tweet has gotten respectively. We can call these values individually by calling the iterable variable, in this case `tweet`, followed with a dot and the field that we want to look at.

`tweet.displayname` returns the display name of the twitter user, while `tweet.followersCount` tells you the amount of followers that the user had at the time of tweeting.

## Pulling specific data from a tweet

In `basescrape.py`, there is a function called `get_tweet` which returns the rawContent of a tweet. Using the content object causes deprecation issues.

Feel free to test grabbing the content of a tweet from any account using the code cell below.

If the tweet is an image, a shortened hyperlink to the tweet will be the output.

In [5]:
NAME = "jack"
tweet = get_tweet(NAME)
print(tweet)

and Google Play Store: https://t.co/1Ve7GIBG0F

and of course, the open web: https://t.co/qXl9xrmtKE (one of many)


You can call multiple of these elements, and print the output in a list to see a list of key statistics for a tweet.

The `get_tweet_data` function does this and returns the date, tweet, and contents in the most recent tweet on a profile.

In [9]:
NAME = "jack"
tweet_data = get_tweet_data(NAME)
print(tweet_data)

[[datetime.datetime(2023, 1, 31, 22, 38, 10, tzinfo=datetime.timezone.utc), 'and Google Play Store: https://t.co/1Ve7GIBG0F\n\nand of course, the open web: https://t.co/qXl9xrmtKE (one of many)', 3273]]


As you can see, the date is in a weird format. However, printing the date separately works fine. This is because the date is being stored as a datetime object, and you can call different aspects if you want to compare them to an existing date time. This is an approach you can take to compare tweet times to times of stock price drops, if you wanted to. Use [this](https://www.listendata.com/2019/07/how-to-use-datetime-in-python.html#id-886a0d) link to learn more about datetime objects.

For the purposes of this project, it is best to convert the datetime object into a string, which can then be broken down into aspects for date and time. THese can then be compared to other strings of dates and times, which might be useful later. The `datetime` class has a function which can this, and below you can see its implementation.
Run the code cell below to import it, then it can be tested with a date time object from a scraped tweet.

(For simplicity, a `get_tweet_date` function was defined that gets the date of the latest tweet from a profile.)

In [20]:
from datetime import datetime
NAME = "jack"
dates = get_tweet_date(NAME)
print(dates)
date = dates.strftime("%d%m%Y")
time = dates.strftime("%H%M%S")
date_type=type(date)
time_type=type(time)
print(f"The date of this tweet is {date} and it is a {date_type}.")
print(f"The time of this tweet is {time} and it is a {time_type}.")


2023-01-31 22:38:10+00:00
The date of this tweet is 31012023 and it is a <class 'str'>.
The time of this tweet is 223810 and it is a <class 'str'>.


## Scraping tweets within a certain timeframe.
Using the basis of the previous functions, the elements can be combined to scrape a profile for a certain number of tweets. It can then later be customized to allow for the number of tweets scraped to be determined, as well as all the tweets tweeted in a certain month. The function `get_thou_tweets` starts by taking the twitter handle as an input, and stores the list of the last 10000 tweets in a csv called `"NAME.csv"` in the `raw-data` folder, where name is the display name of the twitter profile. The data is stored as a pandas dataframe, with the rows representing the date, time, content, and view count of the tweet respectively. The dataframe is the return for further analysis and plotting.

In [7]:
NAME = "elonmusk"

tweets_df = get_thou_tweets(NAME)

You may have noticed that this took a very long time, and if we are planning to scrape tweets across longer time periods, then the number of tweets being stored has to be reduced for time and space reasons. One way that this can be done is by only appending tweets that meet certain parameters set.

One of these parameters is the month that the tweet happens in. The function `get_tweets_in_month` scans the same 10000 tweet period, but this time only tweets within a specifed month are saved to the csv. This data is viewable from the `month-data` folder.

Note: This function differs to the previous ones in a few ways. The `statusesCount` (total tweets) value needs to be grabbed freom an object within the `user` object, so that the loop can be broken out of if the loop index surpasses it. The position of the loop conditions changes, and the additional object call takes place at the start of the for loop.

In [4]:
NAME = "elonmusk"

tweets_df = get_tweets_in_month(NAME, "")

This still takes a long time, so when working with this data it may be best to work on a pre-existing csv, and place the new edited data onto a separtae file, that has everything we are looking for. The dataframe can then be scraped through row by row, with only specific dates and/or times being included. This drastically decreases the computational workload, and allows us to map the instance of a tweet to any graph we want.

## Excluding dates and times
In some cases, you may want to exclude all tweets tweeted past a certain time, as they are not relevant. To make the target datafile as manageable as possible, a number of truncations and assumptions need to be made about the tweets being scraped.

### Truncating tweets after-hours

Some tweets are not always seen by everyone, and they may not even have a link to adjecent charts that we may be interested in. It can therefore be useful to only store tweets tweeted before and after certain periods.

## Generating raw data

Now we have an understanding of all the different aspects of the scraping library, and you have a basic idea of some of the different ways to filter the tweets you are interested in. For the purposes of this project, it is best to scrape all the tweets, and then filter down the data in a csv file to look at parts we are interested in.

The function `get_all_tweets` scrapes through a user's entire profile for all tweets, and stores the data in a csv with the name as their twitter handle.

Note: This can take an extremely long time to run.

In [ ]:
NAME = "elonmusk"
tweets = get_all_tweets(NAME)


For the sake of this data story, it is safe to ignore tweets that took place before around 2019, as the tweets made prior to this were to a much smaller audience, and were tweeted during a time where tweets were under less scrutiny. The `get_tweets_after` function takes the handle and target year, and returns all the tweets after that year.

In [6]:
NAME = "elonmusk"
year = "2017"
tweets = get_tweets_after(NAME,year)

In [4]:
NAME = "elonmusk"
YEAR = "2013"
tweets = get_tweets_on_year(NAME,YEAR)
tweets.loc[:, 'like count'].mean()

In [5]:
tweets.loc[:, 'like count'].mean()

277.3980343980344

Now all the data needed is in a csv, it can be manipulated based on other parameters such as stock data.